<a href="https://colab.research.google.com/github/markpairdha/Face-mask-detection/blob/main/Face_Mask_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import keras
import keras.backend as k
from keras.layers import Conv2D,MaxPooling2D,SpatialDropout2D,Flatten,Dropout,Dense
from keras.models import Sequential,load_model
from keras.optimizers import Adam
from keras.preprocessing import image
import cv2
import datetime

In [3]:
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [4]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/Colab Notebooks/Face-Mask-Detection/train',
        target_size=(150,150),
        batch_size=16 ,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/Colab Notebooks/Face-Mask-Detection/test',
        target_size=(150,150),
        batch_size=16,
        class_mode='binary')

Found 1314 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


In [5]:
model_saved=model.fit_generator(
        training_set,
        epochs=10,
        validation_data=test_set,
        )
model.save('mymodel.h5',model_saved)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
83/83 [==============================] - 482s 5s/step - loss: 0.5009 - accuracy: 0.7232 - val_loss: 0.2778 - val_accuracy: 0.8866
Epoch 2/10
83/83 [==============================] - 14s 168ms/step - loss: 0.1997 - accuracy: 0.9346 - val_loss: 0.1470 - val_accuracy: 0.9433
Epoch 3/10
83/83 [==============================] - 14s 168ms/step - loss: 0.0895 - accuracy: 0.9646 - val_loss: 0.1022 - val_accuracy: 0.9588
Epoch 4/10
83/83 [==============================] - 14s 172ms/step - loss: 0.1072 - accuracy: 0.9554 - val_loss: 0.1096 - val_accuracy: 0.9639
Epoch 5/10
83/83 [==============================] - 15s 181ms/step - loss: 0.1094 - accuracy: 0.9546 - val_loss: 0.1175 - val_accuracy: 0.9588
Epoch 6/10
83/83 [==============================] - 15s 175ms/step - loss: 0.0547 - accuracy: 0.9783 - val_loss: 0.1065 - val_accuracy: 0.9691
Epoch 7/10
83/83 [==============================] - 15s 181ms/step - loss: 0.0464 - accuracy: 0.9849 - val_loss: 0.0815 - val_accuracy: 0.9691
E

In [6]:
#To test for individual images
mymodel=load_model('/content/drive/MyDrive/Colab Notebooks/Face-Mask-Detection/mymodel.h5')
test_image=image.load_img(r'/content/drive/MyDrive/Colab Notebooks/Face-Mask-Detection/with-mask.jpeg',target_size=(150,150,3))
test_image
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
mymodel.predict(test_image)[0][0]

0.0

In [9]:
mymodel=load_model('/content/drive/MyDrive/Colab Notebooks/Face-Mask-Detection/mymodel.h5')
test_image1=image.load_img('/content/drive/MyDrive/Colab Notebooks/Face-Mask-Detection/no_mask.jpeg',target_size=(150,150,3))
test_image1
test_image1=image.img_to_array(test_image1)
test_image1=np.expand_dims(test_image1,axis=0)
mymodel.predict(test_image1)[0][0]

1.0

In [30]:
# IMPLEMENTING LIVE DETECTION OF FACE MASK

mymodel=load_model('/content/drive/MyDrive/Colab Notebooks/Face-Mask-Detection/mymodel.h5')

cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier('/content/drive/MyDrive/Colab Notebooks/Face-Mask-Detection/haarcascade_frontalface_default.xml')

while cap.isOpened():
    _,img=cap.read()
    face=face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4)
    for(x,y,w,h) in face:
        face_img = img[y:y+h, x:x+w]
        cv2.imwrite('temp.jpg',face_img)
        test_image=image.load_img('temp.jpg',target_size=(150,150,3))
        test_image=image.img_to_array(test_image)
        test_image=np.expand_dims(test_image,axis=0)
        pred=mymodel.predict(test_image)[0][0]
        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,'NO MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.putText(img,'MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
        datet=str(datetime.datetime.now())
        cv2.putText(img,datet,(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
          
    cv2.imshow('img',img)
    
    if cv2.waitKey(1)==ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()